# Transformation

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import sklearn
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler,OrdinalEncoder
from sklearn import set_config
set_config(transform_output="pandas")

print(sklearn.__version__)
os.chdir("C:/Users/diego/Desktop/tangelo/")

1.3.0


## Reading & Merging

In [2]:
#Application Records
app_df = pd.read_csv("./data/application_record.csv")
app_df['OCCUPATION_TYPE'].fillna('Other', inplace=True)
app_df.head()


,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Other,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,Other,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
#Credit Records
cred_df=pd.read_csv("./data/credit_record.csv")
# We ony want the last month of information
cred_df=cred_df.groupby('ID').agg({'MONTHS_BALANCE': 'max', 'STATUS': 'first'}).reset_index()
cred_df.drop("MONTHS_BALANCE",axis=1,inplace=True)
# Filter dataset, we only want the IDs that has information in the app_df.
cred_df = cred_df[cred_df['ID'].isin(app_df.ID.unique())]
#People with no loans or that have paid off will be catalogized as 0 (No Risk).
cred_df['STATUS'] = cred_df['STATUS'].replace(['X', 'C'], '0')
cred_df['STATUS'] = cred_df['STATUS'].astype(int)
cred_df["target"] = cred_df["STATUS"].apply(lambda x: 0 if x <= 0 else 1)
cred_df.drop("STATUS",axis=1,inplace=True)
cred_df.head()

,ID,target
3231,5008804,0
3232,5008805,0
3233,5008806,0
3234,5008808,0
3235,5008809,0


In [4]:
df = pd.merge(app_df,cred_df, on ="ID")
df.columns = ('User_id',
              'Gender',
              'Car',
              'Realty',
              'children_count',
              'income_amount',
              'income_type',
              'education_type',
              'Family_status',
              'Housing_type',
              'Days_birth',
              'Days_employed',
              'Mobile',
              'Work_phone',
              'Phone',
              'Email',
              'Occupation_type',
              'Count_family_members',
              'Target'
              )

## Cleaning

In [5]:
#First we transform the variable of days-employed into years
df["Years_Employed"]=abs((df["Days_employed"]/365)).round()
df["Age"]=abs((df["Days_birth"]/365)).round()
df.drop(["Days_employed","Days_birth"],axis=1,inplace=True)

In [6]:
#Searching up a bit I found that Turkey has the lowest retirement age in 2023 with 45 years. So the people above this age and with >100 years employed will be flagged as 'Pensioner'.
df.loc[(df['Age'] >= 45) & (df['Years_Employed'] >= 100), 'Occupation_type'] = 'Pensioner'
df.loc[df['income_type']== "Pensioner", 'Occupation_type'] = 'Pensioner'

#We will keep only the cases where individual's reported age is at leats 15 years higher than the years employed and are not pensioners. (Assuming people can work from 15 years old)
Aux = df[df['income_type'] == 'Pensioner']
df = df[(df['Age'] >= df['Years_Employed'] + 15) & (df['Occupation_type'] != 'Pensioner')]
df = pd.concat([df, Aux])
df.head()

,User_id,Gender,Car,Realty,children_count,income_amount,income_type,education_type,Family_status,Housing_type,Mobile,Work_phone,Phone,Email,Occupation_type,Count_family_members,Target,Years_Employed,Age
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,1,0,0,Other,2.0,0,12.0,33.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,1,1,0,0,Other,2.0,0,12.0,33.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,1,0,0,0,Security staff,2.0,0,3.0,59.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,0,8.0,52.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,0,1,1,Sales staff,1.0,0,8.0,52.0


- User_id                   `ignore`
- Gender                   `One-Hot Encoding`
- Car                      `One-Hot Encoding`
- Realty                   `One-Hot Encoding`
- Mobile                    `One-Hot Encoding`
- Work_phone                `One-Hot Encoding`
- Phone                     `One-Hot Encoding`
- Email                     `One-Hot Encoding`
- education_type           `Ordinal`
- income_type              `Categorical`
- Family_status            `Categorical`
- Housing_type             `Categorical`
- Occupation_type          `Categorical`
- Count_family_members     `Numerical`
- Years_Employed          `Numerical`
- Age                     `Numerical`
- children_count           `Numerical`
- income_amount            `Numerical`
- Target                    int64

## Transformation Pipeline

In [40]:
# List of categorical features
categorical_features = [
    'Gender', 'Car', 'Realty', 'Mobile', 'Work_phone', 'Phone', 'Email',
     'income_type', 'Family_status', 'Housing_type', 'Occupation_type'
]

# List of numerical features
numerical_features = [
    'Count_family_members', 'Years_Employed', 'Age', 'children_count', 'income_amount'
]

# Mapping of education_type values to ordinal values
education_mapping = [
    'Incomplete higher',
    'Lower secondary',
    'Secondary / secondary special',
    'Higher education',
    'Academic degree'
]
# Select the target variable
target_variable = 'Target'

# Create transformers for categorical and numerical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=False))
])

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Create a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('cat', categorical_transformer, categorical_features),
    ('num', numerical_transformer, numerical_features),
    ('education', OrdinalEncoder(categories=[education_mapping]), ['education_type'])
], remainder='passthrough')

# Create the final pipeline including the preprocessor
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Fit and transform the data
df_transformed= pipeline.fit_transform(df.drop("User_id",axis=1))

#Remove some columns duplicated
df_transformed.drop(['cat__Gender_F', 'cat__Car_N', 'cat__Realty_N', 'cat__Mobile_1',
       'cat__Work_phone_1', 'cat__Phone_1', 'cat__Email_0'],axis=1,inplace=True)
df_transformed.head()
df_transformed.to_csv("df_trans.csv")